In [1]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.close_figures=False 

from ANEMO import ANEMO
from ANEMO import read_edf

import os
import pickle
import numpy as np

In [2]:
sujets = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
times = ['2017-10-23_100057','2017-09-26_095637', '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250', '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313', '2017-11-08_150410','2017-11-17_172706']

In [3]:
list_fct = {#'velocity':'fct_velocity',
            'velocity':'fct_velocity_sigmo',
            #'velocity':'fct_velocity_line',
            #'position':'fct_position',
            #'saccade':'fct_saccade'
           }

In [4]:
num_suj = 1

In [5]:
list_param_enre = {}
list_param_enre['fct_velocity'] = ['start_anti', 'a_anti', 'latency', 'tau', 'steady_state']
list_param_enre['fct_velocity_sigmo'] = ['start_anti', 'a_anti', 'latency', 'ramp_pursuit', 'steady_state']
list_param_enre['fct_velocity_line'] = ['start_anti', 'a_anti', 'latency', 'ramp_pursuit', 'steady_state']
list_param_enre['fct_position'] = ['start_anti', 'a_anti', 'latency', 'tau', 'steady_state']
list_param_enre['fct_saccade'] =  ['T0', 't1', 't2', 'tr', 'x_0', 'x1', 'x2', 'tau']

In [6]:
for num_suj in range(len(sujets)) :

    print(sujets[num_suj], '\n-------------------------------------------')
    
    file = os.path.join('data', 'enregistrement_%s_%s.pkl'%(sujets[num_suj], times[num_suj]))
    with open(file, 'rb') as fichier :
        param_exp = pickle.load(fichier, encoding='latin1')
    resultats = os.path.join('data', 'enregistrement_%s_%s.asc'%(sujets[num_suj], times[num_suj]))
    data = read_edf(resultats, 'TRIALID')

    Plot = ANEMO.Plot(param_exp)
    
            
    

    for fct in list_fct.keys():
    
        try :
            file = os.path.join('parametre/%s/param_Fit_%s_%s.pkl'%(list_fct[fct], sujets[num_suj], list_fct[fct]))
            with open(file, 'rb') as fichier :
                Full_param_fit = pickle.load(fichier, encoding='latin1')
            print(list_fct[fct], 'OK param')
        except :
            Full_param_fit = None
            print(list_fct[fct], 'No param')
    
        print('list_delete\n--------------------------------')
        try :
            with open('parametre/Delete/%s/Delete_list_BadTrials_%s_%s.pkl'%(list_fct[fct], sujets[num_suj], list_fct[fct]), 'rb') as fichier:
                f = pickle.load(fichier, encoding='latin1')
            print(f)
            list_delete = f

        except : list_delete = None ; print('No file!')
    
    
        for block in range(param_exp['N_blocks']) :

            trials = list_delete[block]
            print(block, trials)
            
            if len(trials)!=0 :

                fig_width=15
                fig, axs = plt.subplots(len(trials), 1, figsize=(fig_width, (fig_width*(len(trials)/2)/1.6180)))

                for t, trial in enumerate(trials) :

                    if len(trials)==1: ax = axs
                    else : ax = axs[t]
                    
                    param_fit_trial = {}
                    for name in list_param_enre[list_fct[fct]] :
                        if type(Full_param_fit[name][block][trial])!= list :
                            if np.isnan(Full_param_fit[name][block][trial]) : param_fit_trial = None ; break
                            else : param_fit_trial[name] = Full_param_fit[name][block][trial]
                        else :     param_fit_trial[name] = Full_param_fit[name][block][trial]
                    #print(param_fit_trial)
                    ax = Plot.generate_fig(ax, data, trial, block, 
                                           show='fit', show_data=fct, equation=list_fct[fct],
                                           N_blocks=param_exp['N_blocks'], N_trials=param_exp['N_trials'],
                                           time_sup=280, step_fit=2, do_whitening=False,
                                           list_param_enre=None, param_fit=param_fit_trial, inde_vars=None,
                                           before_sacc=5, after_sacc=15, stop_search_misac=None,
                                           filt=None, cutoff=30, sample_rate=1000,
                                           show_pos_sacc=True, plot_detail=None,
                                           show_target=False, show_num_trial=True, write_step_trial=False,
                                           title='', c='k', fig=fig, out=None, report=None,
                                           fig_width=fig_width, t_label=20, t_text=14) 


                plt.tight_layout()
                plt.subplots_adjust(hspace=0)
                plt.savefig('figures/Delete/%s/Delete_list_BadTrials_%s_%s_%s.pdf'%(list_fct[fct], sujets[num_suj], list_fct[fct], block+1))
                plt.close()
                print('fig save in : figures/Delete/%s/Delete_list_BadTrials_%s_%s_%s.pdf'%(list_fct[fct], sujets[num_suj], list_fct[fct], block+1))
                
        print('\n\n')

AM 
-------------------------------------------
fct_velocity_sigmo OK param
list_delete
--------------------------------
[[22, 82, 94], [11, 17, 134, 153, 161, 168, 170, 177], [0, 41, 47, 115]]
0 [22, 82, 94]
fig save in : figures/Delete/fct_velocity_sigmo/Delete_list_BadTrials_AM_fct_velocity_sigmo_1.pdf
1 [11, 17, 134, 153, 161, 168, 170, 177]
fig save in : figures/Delete/fct_velocity_sigmo/Delete_list_BadTrials_AM_fct_velocity_sigmo_2.pdf
2 [0, 41, 47, 115]
fig save in : figures/Delete/fct_velocity_sigmo/Delete_list_BadTrials_AM_fct_velocity_sigmo_3.pdf



BMC 
-------------------------------------------
fct_velocity_sigmo OK param
list_delete
--------------------------------
[[54, 76, 125, 130, 156], [57, 66, 71, 126, 128, 129, 137], [79, 109, 137]]
0 [54, 76, 125, 130, 156]
fig save in : figures/Delete/fct_velocity_sigmo/Delete_list_BadTrials_BMC_fct_velocity_sigmo_1.pdf
1 [57, 66, 71, 126, 128, 129, 137]
fig save in : figures/Delete/fct_velocity_sigmo/Delete_list_BadTrials_BMC_fc

fig save in : figures/Delete/fct_velocity_sigmo/Delete_list_BadTrials_YK_fct_velocity_sigmo_2.pdf
2 [1, 3, 12, 22, 26, 40, 44, 47, 51, 58, 59, 62, 69, 71, 73, 75, 76, 78, 81, 89, 92, 94, 96, 105, 107, 108, 114, 117, 122, 123, 125, 131, 137, 139, 141, 150, 172, 191, 192, 197, 198]
fig save in : figures/Delete/fct_velocity_sigmo/Delete_list_BadTrials_YK_fct_velocity_sigmo_3.pdf



